In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from downhole.survey import Survey, DownholeSurvey

from downhole.examples.ex_survey import *

s = Survey()

import pandas as pd

In [ ]:
survey_df = pd.read_excel('.\\Test Spreadsheets\\Survey.xlsx', index_col=0)
collar_df = pd.read_excel('.\\Test Spreadsheets\\Collar.xlsx', index_col=0)

In [ ]:
holes = list(pd.unique(collar_df.index))
for hole in holes:
    try:
        if len(survey_df.loc[[hole]]) == 1:
            print(f"No Downhole Survey {hole}, collar info used.")
            ds = DownholeSurvey(
                [collar_df.at[hole, 'Azimuth_GridN'],collar_df.at[hole, 'Azimuth_GridN']],
                [collar_df.at[hole, 'DIP'],collar_df.at[hole, 'DIP']],
                [0,collar_df.at[hole, 'TotalDepth_m']],
                collar_df.at[hole, 'UTM_N83_EASTING'],
                collar_df.at[hole, 'UTM_N83_NORTHING'],
                collar_df.at[hole, 'UTM_N83_ELEV_MSL'], hole
            )
        else:
            azm_data = list(survey_df.loc[[hole]]['AzimGridN'])
            dip_data = list(survey_df.loc[[hole]]['Final_Dip'])
            depth_data = list(survey_df.loc[[hole]]['Depth_m'])
            
            if collar_df.at[hole, 'TotalDepth_m'] > (depth_data[-1] + 25):
                difference = collar_df.at[hole, 'TotalDepth_m'] - depth_data[-1]
                print(f"Collar depth > survey depth by 25m for {hole}, interval added. {difference}")
                azm_data.insert(len(azm_data),azm_data[-1])
                dip_data.insert(len(dip_data),dip_data[-1])
                depth_data.insert(len(depth_data),collar_df.at[hole, 'TotalDepth_m'])
            
            
            if min(survey_df.loc[[hole]]['Depth_m']) > 9:
                print(min(survey_df.loc[[hole]]['Depth_m']),azm_data[0], dip_data[0])
                print(collar_df.at[hole, 'Azimuth_GridN'], collar_df.at[hole, 'DIP'])
                print(f"No Depth=0.00 reading for {hole}, interval added.")
                azm_data.insert(0,azm_data[0])
                dip_data.insert(0,dip_data[0])
                depth_data.insert(0,0.00)
            
            ds = DownholeSurvey(
                azm_data,
                dip_data,
                depth_data,
                collar_df.at[hole, 'UTM_N83_EASTING'],
                collar_df.at[hole, 'UTM_N83_NORTHING'],
                collar_df.at[hole, 'UTM_N83_ELEV_MSL'], hole
            )
        

    except KeyError:
        print(f"No Downhole Survey or record of {hole}, collar info used.")
        ds = DownholeSurvey(
            [collar_df.at[hole, 'Azimuth_GridN'],collar_df.at[hole, 'Azimuth_GridN']],
            [collar_df.at[hole, 'DIP'],collar_df.at[hole, 'DIP']],
            [0,collar_df.at[hole, 'TotalDepth_m']],
            collar_df.at[hole, 'UTM_N83_EASTING'],
            collar_df.at[hole, 'UTM_N83_NORTHING'],
            collar_df.at[hole, 'UTM_N83_ELEV_MSL'], hole
        )
    
    s.add_downhole_survey(ds)


In [ ]:
s.export_shapefile()

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from math import sqrt

In [ ]:
intervals = s.downhole_surveys[25].min_curv.survey_intervals
intervals

In [ ]:
s.downhole_surveys[25].min_curv.sample_interval(5,25)

In [ ]:
s.downhole_surveys[25].min_curv.sample_point(2)
xs = [i[1][0] for i in intervals]
ys = [i[1][1] for i in intervals]
zs = [i[1][2] for i in intervals]

xs1 = [i[1][0] for i in s.downhole_surveys[25].min_curv.sample_interval(100,156)]
ys1 = [i[1][1] for i in s.downhole_surveys[25].min_curv.sample_interval(100,156)]
zs1 = [i[1][2] for i in s.downhole_surveys[25].min_curv.sample_interval(100,156)]


#x = [s.downhole_surveys[25].min_curv.sample_point(i)[1][0] for i in range(0,200,10)]
#y = [s.downhole_surveys[25].min_curv.sample_point(i)[1][1] for i in range(0,200,10)]
#z = [s.downhole_surveys[25].min_curv.sample_point(i)[1][2] for i in range(0,200,10)]

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.plot(xs,ys,zs, linewidth=1)
ax.plot(xs1,ys1,zs1, linewidth=4)

#ax.scatter(x,y,z)


In [ ]:
s.downhole_surveys[15].min_curv.sample_point(223)